In [ ]:
from golden import *
import pandas as pd
import numpy as np

In [ ]:
# Upload de arquivos:
# df_neurotech = pd.read_csv('/mnt/c/Users/anderson.paiva/Documents/03.3 Squad Saúde/Projeto Neurotech/poc2_escorada_petlove.csv')

In [ ]:
# Descritivo do arquivo
# df_neurotech.info()

In [ ]:
# Iniciando o client para acessar o BQ
client_bq = config_client('prod')

# Verificando se o client está conectado
check_client_connection(client_bq)

# Declarando a query que vamos utilizar
query = """
WITH
  realizado AS ( -- Base 'realizado' para o último mês
  SELECT
    sku,
    filial,
    EXTRACT(MONTH FROM data) AS mes,
    EXTRACT(YEAR FROM data) AS ano,
    SUM(quantidade) AS Qtde_Fat_Real,
    SUM(receita_bruta_produto) AS Receita_Real
  FROM `petlove-dataeng-prod-01.op_octopus.supply_chain_historico_vendas`
  WHERE 1=1 
    AND data BETWEEN '2024-09-01' AND '2024-09-30'
  GROUP BY
    sku,
    filial,
    mes,
    ano
  ),
  gerada AS ( -- Previsões geradas pelo modelo para o último mês (setembro)
  SELECT
    sku,
    filial,
    modelo,
    -- Nova coluna modelo_abreviado
    CASE
      WHEN modelo LIKE 'mape%' THEN 'vmd'
      WHEN modelo IN ('DIARIO', 'SEMANAL')
      THEN 'algoritmo' END AS modelo_abreviado,
    EXTRACT(MONTH FROM data) AS mes,
    EXTRACT(YEAR FROM data) AS ano,
    SUM(previsao) AS previsao_gerada,
    refdate AS refdate_gerada
  FROM `supply_chain_analytics.previsoes_sku_filial_versionado`
  WHERE 1=1
    AND data BETWEEN DATE_TRUNC(DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH), MONTH)
    AND LAST_DAY(DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH))
    AND previsao > 0
    AND refdate = (
      SELECT
        MAX(refdate)
      FROM `supply_chain_analytics.previsoes_sku_filial_versionado`
      WHERE 1=1
        AND EXTRACT(MONTH FROM refdate) = EXTRACT(MONTH FROM DATE_SUB(CURRENT_DATE(), INTERVAL 2 MONTH))
        AND EXTRACT(YEAR FROM refdate) = EXTRACT(YEAR FROM DATE_SUB(CURRENT_DATE(), INTERVAL 2 MONTH))
      )
  GROUP BY
    sku,
    filial,
    refdate_gerada,
    modelo,
    modelo_abreviado,
    mes,
    ano
  ),
  revisada AS ( -- Previsões revisadas pelo time de planejamento
  WITH
    tabela_previsao_revisada AS (
    SELECT
      sku,
      data,
      filial,
      CASE
        WHEN refdate = '2024-09-02'
        THEN '2024-08-31'
        ELSE refdate END AS refdate,
      previsao
    FROM `supply_chain_analytics.best_prev_revisado_sentinela`
    WHERE 1=1
      AND refdate >= DATE_TRUNC(DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH), MONTH) - INTERVAL 30 DAY
    )
  SELECT
    sku,
    filial,
    refdate AS refdate_revisada,
    EXTRACT(MONTH FROM data) AS mes,
    EXTRACT(YEAR FROM data) AS ano,
    SUM(previsao) AS previsao_revisada
  FROM tabela_previsao_revisada
  WHERE 1=1
    AND data BETWEEN DATE_TRUNC(DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH), MONTH)
    AND LAST_DAY(DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH))
    AND previsao > 0
    AND refdate = (
      SELECT
        MAX(refdate)
      FROM tabela_previsao_revisada
      WHERE 1=1
        AND EXTRACT(MONTH FROM refdate) = EXTRACT(MONTH FROM DATE_SUB(CURRENT_DATE(), INTERVAL 2 MONTH))
        AND EXTRACT(YEAR FROM refdate) = EXTRACT(YEAR FROM DATE_SUB(CURRENT_DATE(), INTERVAL 2 MONTH))
      )
  GROUP BY
    sku,
    filial,
    refdate_revisada,
    mes,
    ano
   ),
  comparacao AS ( -- Comparação das previsões com o realizado
  SELECT
    r.sku,
    r.filial,
    r.mes,
    r.ano,
    r.Qtde_Fat_Real,
    r.Receita_Real,
    g.previsao_gerada,
    g.refdate_gerada,
    g.modelo,
    g.modelo_abreviado,
    rev.previsao_revisada,
    rev.refdate_revisada,
    CASE -- Verificação de previsões nulas
      WHEN g.previsao_gerada IS NULL AND rev.previsao_revisada IS NOT NULL THEN 'Sem previsão gerada e com previsão revisada'
      WHEN g.previsao_gerada IS NOT NULL AND rev.previsao_revisada IS NULL THEN 'Sem previsão revisada e com previsão gerada'
      WHEN g.previsao_gerada IS NULL AND rev.previsao_revisada IS NULL THEN 'Sem previsão revisada e sem previsão gerada' -- Comparação normal entre gerada e revisada quando ambas não são nulas
      WHEN ABS(r.Qtde_Fat_Real - g.previsao_gerada) < ABS(r.Qtde_Fat_Real - rev.previsao_revisada) THEN 'Gerada é melhor'
      WHEN ABS(r.Qtde_Fat_Real - g.previsao_gerada) > ABS(r.Qtde_Fat_Real - rev.previsao_revisada) THEN 'Revisada é melhor' -- Caso empate, quando ambas AS previsões não são nulas
      ELSE 'Empate' END AS melhor_previsao,
    (SUM(r.Qtde_Fat_Real) - SUM(g.previsao_gerada)) / NULLIF(SUM(r.Qtde_Fat_Real), 0) AS mpe_gerada,
    (SUM(r.Qtde_Fat_Real) - SUM(rev.previsao_revisada)) / NULLIF(SUM(r.Qtde_Fat_Real), 0) AS mpe_revisada
  FROM realizado r
  LEFT JOIN gerada g
    ON r.sku = g.sku
    AND r.filial = g.filial
    AND r.mes = g.mes
  LEFT JOIN revisada rev
    ON r.sku = rev.sku
    AND r.filial = rev.filial
    AND r.mes = rev.mes
  GROUP BY
    r.sku,
    r.filial,
    r.Qtde_Fat_Real,
    r.mes,
    r.ano,
    r.Receita_Real,
    g.refdate_gerada,
    g.previsao_gerada,
    g.modelo,
    g.modelo_abreviado,
    rev.refdate_revisada,
    rev.previsao_revisada
  ) -- Seleção dos dados desejados
SELECT
  sku,
  filial,
  mes,
  ano,
  Qtde_Fat_Real,
  Receita_Real,
  previsao_gerada,
  previsao_revisada,
  refdate_gerada,
  refdate_revisada,
  modelo,
  modelo_abreviado,
  melhor_previsao,
  (CASE
      WHEN previsao_gerada>0 AND Qtde_Fat_Real=0 THEN '00. Sem realizado'
      WHEN mpe_gerada <-1 THEN '01. Menor -100%'
      WHEN mpe_gerada >= -1 AND mpe_gerada < -0.8 THEN '02. -100% a -80%'
      WHEN mpe_gerada >= -0.8 AND mpe_gerada < -0.6 THEN '03. -80% a -60%'
      WHEN mpe_gerada >= -0.6 AND mpe_gerada < -0.4 THEN '04. -60% a -40%'
      WHEN mpe_gerada >= -0.4 AND mpe_gerada < -0.2 THEN '05. -40% a -20%'
      WHEN mpe_gerada >= -0.2 AND mpe_gerada < -0.1 THEN '06. -20% a -10%'
      WHEN mpe_gerada >= -0.1 AND mpe_gerada < 0.1 THEN '07. -10% a 10%'
      WHEN mpe_gerada >= 0.1 AND mpe_gerada < 0.2 THEN '08. 10% a 20%'
      WHEN mpe_gerada >= 0.2 AND mpe_gerada < 0.4 THEN '09. 20% a 40%'
      WHEN mpe_gerada >= 0.4 AND mpe_gerada < 0.6 THEN '10. 40% a 60%'
      WHEN mpe_gerada >= 0.6 AND mpe_gerada < 0.8 THEN '11. 60% a 80%'
      WHEN mpe_gerada >= 0.8 AND mpe_gerada < 1 THEN '12. 80% a 100%'
      WHEN mpe_gerada >= 1 THEN '13. Maior 100%'
      WHEN (previsao_gerada=0 OR previsao_gerada IS NULL) AND Qtde_Fat_Real>0 THEN '14. Sem previsão' END) AS faixa_erro_previsao_gerada,
  (CASE
      WHEN previsao_revisada>0 AND Qtde_Fat_Real=0 THEN '00. Sem realizado'
      WHEN mpe_revisada <-1 THEN '01. Menor -100%'
      WHEN mpe_revisada >= -1 AND mpe_revisada < -0.8 THEN '02. -100% a -80%'
      WHEN mpe_revisada >= -0.8 AND mpe_revisada < -0.6 THEN '03. -80% a -60%'
      WHEN mpe_revisada >= -0.6 AND mpe_revisada < -0.4 THEN '04. -60% a -40%'
      WHEN mpe_revisada >= -0.4 AND mpe_revisada < -0.2 THEN '05. -40% a -20%'
      WHEN mpe_revisada >= -0.2 AND mpe_revisada < -0.1 THEN '06. -20% a -10%'
      WHEN mpe_revisada >= -0.1 AND mpe_revisada < 0.1 THEN '07. -10% a 10%'
      WHEN mpe_revisada >= 0.1 AND mpe_revisada < 0.2 THEN '08. 10% a 20%'
      WHEN mpe_revisada >= 0.2 AND mpe_revisada < 0.4 THEN '09. 20% a 40%'
      WHEN mpe_revisada >= 0.4 AND mpe_revisada < 0.6 THEN '10. 40% a 60%'
      WHEN mpe_revisada >= 0.6 AND mpe_revisada < 0.8 THEN '11. 60% a 80%'
      WHEN mpe_revisada >= 0.8 AND mpe_revisada < 1 THEN '12. 80% a 100%'
      WHEN mpe_revisada >= 1 THEN '13. Maior 100%'
      WHEN (previsao_revisada=0 OR previsao_revisada IS NULL) AND Qtde_Fat_Real>0 THEN '14. Sem previsão' END) AS faixa_erro_previsao_revisada
FROM comparacao
"""

query_bytes(query, client_bq)

In [ ]:
df = run_query(query, client_bq)

In [ ]:
df.info()

In [ ]:
df_group_1 = df_erros_prev.groupby(['nm_filial']).agg({
    'previsao_st': ['sum'],
    'previsao_py': ['sum'],
    'quantidade': ['sum'],
})

In [ ]:
print(df)

In [ ]:
# Merge de bases
# df_uniao = pd.merge(df_neurotech, right = df_custos_medicos, how = 'left', left_on = 'ID_EVENTO', right_on = 'id_atendimento')

In [ ]:
# Drop de colunas
# df_uniao = df_uniao.drop(['id_atendimento', 'numero_atendimento', 'data_origem_date', 'nome_procedimento'], axis = 1)

In [ ]:
# Renomeando colunas
# df_uniao = df_uniao.rename(columns = {'PREV ANALISADO': 'FL_JA_ANALISADO', 'status_novo': 'STATUS', 'vl_bruto': 'VL_REPASSE', 'motivo_glosa_desc': 'MOTIVO_GLOSA', 'fl_edicao_procedimento':'FL_EDICAO_PROCEDIMENTO', 'diff_apos_edicao': 'DIFF_APOS_EDICAO', 'status_glosa':'STATUS_GLOSA'})

In [ ]:
save_to_csv(df, path = '/mnt/c/Users/anderson.paiva/Downloads', file_name = 'base_prev')

------------------------